### Parte 1: Preparar embedding de query 

Usando tecnología similar a la actividad de RAG, podemos pasar una pregunta ficticia a un vector

In [3]:
import os

from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name="intfloat/multilingual-e5-base", device="cpu", prompt="passage: "):
        self.model = SentenceTransformer(model_name, device=device)
        self.prompt = prompt

    def embed_documents(self, texts):
        # Add prompt to each text (E5 model expects it!)
        texts = [self.prompt + text for text in texts]
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text):
        # For queries, E5 expects "query: ..."
        return self.model.encode(["query: " + text], normalize_embeddings=True)[0].tolist()


In [4]:
model_name = "intfloat/multilingual-e5-base"
device = "cpu"  # or "mps" if using Apple GPU

# ✅ Load model
model = SentenceTransformerEmbeddings(model_name, device=device)

# ✅ Your prompts
prompts = {"query": "query: ", "passage": "passage: "}
default_prompt_name = "passage"

/Users/juan/downloads/ENTER/envs/st_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
vector = model.embed_query("proyecto para la protección de humedales")
str(vector)

'[-0.006053100805729628, 0.03398176655173302, -0.03050241619348526, 0.015499376691877842, 0.055578071624040604, -0.013171728700399399, 0.009180222637951374, -0.03328968584537506, 0.04451398923993111, 0.03496410697698593, -0.024194825440645218, 0.03037731535732746, 0.12562336027622223, 0.019807597622275352, -0.05903932452201843, 0.0017877001082524657, 0.017924077808856964, -0.03379158675670624, 0.04403260722756386, -0.031189436092972755, 0.03690820559859276, -0.016695864498615265, 0.07041022181510925, -0.04371967539191246, 0.054779309779405594, -0.03472517430782318, 0.0032929962035268545, 0.027863601222634315, -0.08468478918075562, 0.0188374612480402, 0.01651543751358986, -0.0003188949194736779, 0.035711079835891724, 0.049595706164836884, 0.04971891641616821, 0.04798947274684906, -0.04636099562048912, -0.04474988952279091, 0.058662474155426025, 0.008966071531176567, 0.013616793788969517, 0.02128332480788231, 0.032382458448410034, -0.01618722826242447, 0.01032314170151949, -0.00589618040

### Parte 2: Hacia un RAG básico con MillenniumDB

In [7]:
import millenniumdb_driver

url = 'http://localhost:1234'
driver = millenniumdb_driver.driver(url)

In [12]:
session = driver.session()

Ahora, buscamos los 10 vectores más cercanos al embedding de la consulta que ya habíamos calculado

In [9]:
query = 'CALL HNSW_TOP_K("example", tensorFloat("'+str(vector)+'"), 10, 10000) YIELD ?object AS ?similarItem, ?distance RETURN *'
result = session.run(query)

Este resultado es un nodo y una distancia. Dado que sabemos la estructura del grafo, vamos a buscar la participación correspondiente a ese nodo

In [10]:
for record in result.values():
    q =  'MATCH ('+str(record[0])+')-[:tieneParticipation]->(?nodo) RETURN ?nodo, ?nodo.texto_principal'
    respuesta = session.run(q)
    print(respuesta.values())

[[GraphNode<partic_141448>, 'La señora HERTZ, doña Carmen (Vicepresidenta).- \n \n Tiene la palabra la diputada Nathalie Castillo .  \n \nLa señora CASTILLO (doña Nathalie).- \n \n Señora Presidenta, por su intermedio quiero saludar al ministro. Qué bueno tenerlo en esta Sala. \nFiu está en todo el territorio. \n(Aplausos) \n \nTambién lo tenemos en nuestros queridos humedales de la Región de Coquimbo. Por eso, saludamos que el proyecto incorpore los humedales costeros entre los lugares en los que se prohíbe el ingreso y tránsito de vehículos motorizados, lo cual es una práctica habitual que se realiza sin ningún tipo de consideración y criterio. \nEl proyecto viene a completar en cierta medida lo que falta para la protección aún incipiente de los humedales de nuestro país. \nSegún los datos del Ministerio del Medio Ambiente, la Región de Coquimbo tiene 1.028 humedales, que desempeñan un papel importantísimo en el mantenimiento de la biodiversidad y en el equilibrio ecológico, de los c

In [13]:
#### esta consulta obtiene todos los textos que han salido embeddings (?object) 
#### en participaciones (?part)
#### de parlamentarios (?parlamentario)
#### pertenecientes al Partido Socialista de Chile. 

consulta_grafo = 'MATCH (?object)-[:tieneParticipation]->(?part), (?parlamentario)-[:enParticipacion]->(?part), (?parlamentario)-[:enPartido]->(?partido) WHERE ?partido.nombre == "Partido Socialista de Chile" RETURN DISTINCT ?part.texto_principal'

In [15]:
#### Ahora podemos filtrar nuestra búsqueda de vectores para que solo tome en 
#### Consideracion nodos que salen de esa consulta (es decir, texto producido 
#### Por parlamentarios del Partido Socialista 

query = 'CALL HNSW_TOP_K("example", tensorFloat("'+str(vector)+'"), 5, 10000) YIELD ?object '+consulta_grafo
result = session.run(query)

In [18]:
#### Imprimimos solo el tercer resultado

print(result.values()[2])

['La señorita  PÉREZ ,   doña Catalina  (Vicepresidenta).- \nTiene la palabra el diputado  Daniel Manouchehri .  \n \nEl señor MANOUCHEHRI.- \n \n Señorita Presidenta, “el camino al infierno está cargado de buenas intenciones”. Este dicho popular describe de buena manera el resultado final del proyecto que crea el Servicio de Biodiversidad y Áreas Protegidas (SBAP). \nEn primer término, este proyecto no protege las áreas silvestres, sino que viene a consolidar la industria en las áreas protegidas. ¿Alguien puede explicar cómo se puede pretender decir que se protegen las áreas si se permite una industria como la salmonera, una industria tóxica que destruye el ecosistema, en los lugares que se dice proteger? \nEn segundo término, esta es la neoliberalización de nuestras áreas silvestres. Esta iniciativa abre la puerta y consolida la privatización de los parques nacionales con concesiones de hasta 50 años. En tiempos en que los parques nacionales deberían ser lugares abiertos a la comunid

In [11]:
session.close()